<a href="https://colab.research.google.com/github/hammaad2002/SimpleASRmodel/blob/main/Simple_CRDNN_based_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets==1.18.3

In [ ]:
#importing necessary modules/packages
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
import torchaudio
from typing import List, Tuple
import torch.optim as optim
from datasets import load_dataset
from tqdm.notebook import tqdm

# <center>**Simple CRDNN model**

R = RNN

C = CNN

D = DNN

In [ ]:
class SimpleASR(nn.Module):
    def __init__(self, labels =29):
        super(SimpleASR, self).__init__()
        
        # Define your layers 7 layer CNN
        self.layer1 = nn.Conv1d(in_channels= 1, 
                                out_channels = 512,
                                kernel_size= 10, stride = 5)
        
        self.layer1_norm = nn.LayerNorm(512)

        self.gelu1 = nn.GELU()

        self.dropout1 = nn.Dropout(p=0.2)
        
        self.layer2 = nn.Conv1d(in_channels= 512, 
                                out_channels = 512,
                                kernel_size= 3, stride = 2)
        
        self.layer2_norm = nn.LayerNorm(512)

        self.gelu2 = nn.GELU()
        
        self.dropout2 = nn.Dropout(p=0.2)

        self.layer3 = nn.Conv1d(in_channels= 512, 
                                out_channels = 512,
                                kernel_size= 3, stride = 2)
        
        self.layer3_norm = nn.LayerNorm(512)

        self.gelu3 = nn.GELU()
        
        self.dropout3 = nn.Dropout(p=0.2)
       
        self.layer4 = nn.Conv1d(in_channels= 512, 
                                out_channels = 512,
                                kernel_size= 3, stride = 2)
        
        self.layer4_norm = nn.LayerNorm(512)

        self.gelu4 = nn.GELU()
        
        self.dropout4 = nn.Dropout(p=0.2)
        
        self.layer5 = nn.Conv1d(in_channels= 512, 
                                out_channels = 512,
                                kernel_size= 3, stride = 2)
        
        self.layer5_norm = nn.LayerNorm(512)

        self.gelu5 = nn.GELU()
        
        self.dropout5 = nn.Dropout(p=0.2)
        
        self.layer6 = nn.Conv1d(in_channels= 512, 
                                out_channels = 512,
                                kernel_size= 3, stride = 2)
        
        self.layer6_norm = nn.LayerNorm(512)

        self.gelu6 = nn.GELU()
        
        self.dropout6 = nn.Dropout(p=0.2)
        
        self.layer7 = nn.Conv1d(in_channels= 512, 
                                out_channels = 512,
                                kernel_size= 2, stride = 2)
        
        self.layer7_norm = nn.LayerNorm(512)

        self.gelu7 = nn.GELU()
        
        self.dropout7 = nn.Dropout(p=0.2)
        
        self.layer8 = nn.LSTM(input_size = 512, hidden_size = 128, 
                             num_layers = 1, bias = True, 
                             batch_first = True, bidirectional=True)
        
        self.gelu8 = nn.GELU()

        self.dropout8 = nn.Dropout(p=0.2)
        
        self.layer9 = nn.LSTM(input_size = 256, hidden_size = 64,  #[batch_size, sequence_length, hidden_size * 2] that's why this hidden size is 4 times less than the input of the next layer
                             num_layers = 1, bias = True, 
                             batch_first = True, bidirectional=True)
        
        self.gelu9 = nn.GELU()

        self.dropout9 = nn.Dropout(p=0.2)
        
        self.layer10 = nn.LSTM(input_size = 128, hidden_size = 32, 
                             num_layers = 1, bias = True, 
                             batch_first = True, bidirectional=True)
        
        self.gelu10 = nn.GELU()

        self.dropout10 = nn.Dropout(p=0.2)

        self.layer11 = nn.LSTM(input_size = 64, hidden_size = 16, 
                             num_layers = 1, bias = True, 
                             batch_first = True, bidirectional=True)
        
        self.gelu11 = nn.GELU()

        self.dropout11 = nn.Dropout(p=0.2)

        self.layer12 = nn.Linear(in_features = 32, out_features = labels)

    def forward(self, x):
        # Define the forward pass
        x = x.unsqueeze(1)

        x = self.layer1(x)
        x = x.transpose(1, 2)
        x = self.layer1_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu1(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = x.transpose(1, 2)
        x = self.layer2_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu2(x)
        x = self.dropout2(x)
        
        x = self.layer3(x)
        x = x.transpose(1, 2)
        x = self.layer3_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu3(x)
        x = self.dropout3(x)

        x = self.layer4(x)
        x = x.transpose(1, 2)
        x = self.layer4_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu4(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        x = x.transpose(1, 2)
        x = self.layer5_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu5(x)
        x = self.dropout5(x)

        x = self.layer6(x)
        x = x.transpose(1, 2)
        x = self.layer6_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu6(x)
        x = self.dropout6(x)

        x = self.layer7(x)
        x = x.transpose(1, 2)
        x = self.layer7_norm(x)
        x = x.transpose(1, 2)
        x = self.gelu7(x)
        x = self.dropout7(x)    # [batch, features, timesteps]

        x = x.transpose(1, 2) # [batch, timesteps, features]
        x = self.layer8(x)
        x = self.gelu8(x[0])
        x = self.dropout8(x)

        x = self.layer9(x)
        x = self.gelu9(x[0])
        x = self.dropout9(x)
        
        x = self.layer10(x)
        x = self.gelu10(x[0])
        x = self.dropout10(x)
        
        x = self.layer11(x)
        x = self.gelu11(x[0])
        x = self.dropout11(x)
        
        x = self.layer12(x)
        return x

In [ ]:
class DataPipeline(object):
    def __init__(self, labels: Tuple, model):
        self.labels = labels
        self.model = model
    
    def zeroPadder(self, audios: List):
        maxLengths = max([audio.shape[1] for audio in audios])
        new = []
        for audio in audios:
          if audio.shape[1] == maxLengths:
            new.append(audio.squeeze())
          else:
            diff = maxLengths - audio.shape[1] # [batch, timesteps]
            padded_x = torch.nn.functional.pad(audio.squeeze(), (0, diff), mode='constant', value=0)
            new.append(padded_x)
        return torch.stack(new, dim = 0)

    def transcription_encoder(self, transcriptions: List[str]):
        labelsDic = {val: i  for i, val in enumerate(self.labels)}
        transcriptions = [[char.replace(" ", "|") for char in transcription] for transcription in transcriptions]
        empty = [[labelsDic[char] for char in transcription] for transcription in transcriptions]
        return empty

    def transcription_padder(self, x):
        max_length = max([len(i) for i in x])
        temp = []
        for i in x:
          if len(i) == max_length:
            temp.append(i)
          else:
            diff = max_length - len(i)
            for _ in range(diff):
              i.append(0)
            temp.append(i)
        return torch.tensor(temp, dtype=torch.float32)
        
    def inference(self, input):
        # Inference method of the model
        blank = 0
        with torch.inference_mode():
          output = self.model(input)
        transc = []
        for i in range(output.shape[0]):
          encodedTrans = torch.argmax(output[i], axis=-1)
          encodedTrans = torch.unique_consecutive(encodedTrans, dim=-1)
          indices = [i for i in encodedTrans if i != blank]
          transc.append("".join([self.labels[i] for i in indices]))
        return transc

In [ ]:
device = 'cpu' if not torch.cuda.is_available() else 'cuda'
print(f"Current available device is '{device}'.")

Current available device is 'cuda'.


In [ ]:
# initializing dictionary and our model
dictionary = ["-" , "|", "'", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
labels = len(dictionary)
print("Total number of labels are: ", labels)


# Create an instance of the model
model = SimpleASR(labels = labels)

Total number of labels are:  29


In [ ]:
pipeline = DataPipeline(dictionary, model)

In [ ]:
#loading the dataset from 'datasets' library
%%capture
timit = load_dataset("timit_asr")

In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [ ]:
timit = timit.remove_columns(['phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'])

In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
total = len(timit['train']['audio']) + 1 # total number of samples

# calculating total number of batches
no_of_batches = (total-1)//4
print("Total number of batches will be:", no_of_batches)

Total number of batches will be: 1155


In [ ]:
# creating batches of audios and transcription
batch_audios = []
batch_transcriptions = []
i_prev = 0
for i in tqdm(range(4 , total, 4)):
    batch_transcriptions.append(timit['train']['text'][i_prev:i])
    batch_audios.append([torchaudio.load(path)[0] for path in timit['train']['file'][i_prev:i]])
    i_prev = i

  0%|          | 0/1155 [00:00<?, ?it/s]

In [ ]:
model = model.to(device)

In [ ]:
use_amp = True

scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

optimizer = optim.Adam(model.parameters(), lr=1e-3)                 # Adam

# Training parameters
num_epochs = 15

# Storing loss
loss_info = []

# Train the model on GPU
for epoch in tqdm(range(num_epochs + 1)):
    
    # Initialize the running loss and metric for the epoch
    running_loss = 0.0

    # Loop over the batches in the data loader
    for i in tqdm(range(no_of_batches),colour="red", leave = False):
        
        padded_audios = pipeline.zeroPadder(batch_audios[i])
        encoded_transcription = pipeline.transcription_encoder(batch_transcriptions[i])
        padded_transcription = pipeline.transcription_padder(encoded_transcription)

        # Forward pass
        with torch.autocast(device_type= 'cuda', dtype=torch.float16, enabled=use_amp):
            output = model(padded_audios.to(device))

            # Softmax Activation for computing logits
            output = F.log_softmax(output, dim=-1)
            output = output.permute(1, 0 ,2) 

            # CTC loss prep
            target_ = padded_transcription
            output_length = torch.tensor([ output.shape[0] for i in range(output.shape[1])], dtype=torch.long).to(device)
            target_length = torch.tensor([len(i) for i in target_], dtype=torch.long).to(device)

            # Computing loss
            loss = F.ctc_loss(output, target_, output_length, target_length, blank=0, reduction='mean', zero_infinity = True) 

        # Backward pass and optimization
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Reset the optimizer gradients
        optimizer.zero_grad(set_to_none=True)
        
        # Update the running loss and metric for the epoch
        running_loss += loss.item()
    
    # Calculate the average loss and metric for the epoch
    epoch_loss = running_loss / total
    loss_info.append(epoch_loss)
    print("-"*25, f"Epoch {epoch + 1}","-"*25)
    print(f"Epoch {epoch+1} CTC loss: {epoch_loss:.9f}")
    
print("Training completed!")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1155 [00:00<?, ?it/s]

------------------------- Epoch 1 -------------------------
Epoch 1 CTC loss: 0.513031853


  0%|          | 0/1155 [00:00<?, ?it/s]

------------------------- Epoch 2 -------------------------
Epoch 2 CTC loss: 0.382312182


...

------------------------- Epoch 13 -------------------------
Epoch 13 CTC loss: 0.362995726


  0%|          | 0/1155 [00:00<?, ?it/s]

------------------------- Epoch 14 -------------------------
Epoch 14 CTC loss: 0.361025097


  0%|          | 0/1155 [00:00<?, ?it/s]

------------------------- Epoch 15 -------------------------
Epoch 15 CTC loss: 0.359495355
Training completed!


In [ ]:
pipeline.inference(pipeline.zeroPadder(batch_audios[5]).to(device))

['the|||a|a||a', 'toe||o|a|n|||', 'the||||a||||', 'the|||a||||']